In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [3]:
recipes = pd.read_csv(r'DataSets\20_edamam_recipes.csv')
recipes.head(2)

,label,images,url,shareAs,yield,dietLabels,healthLabels,cautions,ingredients,calories,cuisineType,mealType,dishType
0,Easy Chicken Biryani,https://edamam-product-images.s3.amazonaws.com...,http://www.mummymishaps.co.uk/2014/03/easy-chi...,http://www.edamam.com/recipe/easy-chicken-biry...,4.0,"['Balanced', 'Low-Sodium']","['Kidney-Friendly', 'Dairy-Free', 'Gluten-Free...","['Sulfites', 'FODMAP']","[{'text': '* 1 onion, chopped finely', 'quanti...",854.472369,['indian'],['lunch/dinner'],['main course']
1,Three-Onion Farro Soup,https://edamam-product-images.s3.amazonaws.com...,https://www.foodnetwork.com/recipes/food-netwo...,http://www.edamam.com/recipe/three-onion-farro...,4.0,"['Balanced', 'High-Fiber']","['Egg-Free', 'Peanut-Free', 'Tree-Nut-Free', '...",['Sulfites'],"[{'text': '4 tablespoons unsalted butter', 'qu...",2268.805938,['eastern europe'],['lunch/dinner'],['soup']


In [4]:
col_names = ['recipe_id', 'label', 'images', 'url', 'shareAs', 'yield', 'dietLabels',
       'healthLabels', 'cautions', 'ingredients', 'calories',
       'cuisineType', 'mealType', 'dishType']
col_names

['recipe_id',
 'label',
 'images',
 'url',
 'shareAs',
 'yield',
 'dietLabels',
 'healthLabels',
 'cautions',
 'ingredients',
 'calories',
 'cuisineType',
 'mealType',
 'dishType']

In [5]:
conn = sqlite3.connect('final_project.db')
c = conn.cursor()
recipes = c.execute(""" SELECT * FROM recipes;""").fetchall()
conn.close()
recipes = pd.DataFrame(recipes, columns=col_names)
recipes.head(2)

,recipe_id,label,images,url,shareAs,yield,dietLabels,healthLabels,cautions,ingredients,calories,cuisineType,mealType,dishType
0,1,Easy Chicken Biryani,https://edamam-product-images.s3.amazonaws.com...,http://www.mummymishaps.co.uk/2014/03/easy-chi...,http://www.edamam.com/recipe/easy-chicken-biry...,4.0,"['Balanced', 'Low-Sodium']","['Kidney-Friendly', 'Dairy-Free', 'Gluten-Free...","['Sulfites', 'FODMAP']","[{'text': '* 1 onion, chopped finely', 'quanti...",854.47236884,['indian'],['lunch/dinner'],['main course']
1,2,Three-Onion Farro Soup,https://edamam-product-images.s3.amazonaws.com...,https://www.foodnetwork.com/recipes/food-netwo...,http://www.edamam.com/recipe/three-onion-farro...,4.0,"['Balanced', 'High-Fiber']","['Egg-Free', 'Peanut-Free', 'Tree-Nut-Free', '...",['Sulfites'],"[{'text': '4 tablespoons unsalted butter', 'qu...",2268.80593766153,['eastern europe'],['lunch/dinner'],['soup']


In [6]:
def clean_list(row):
    return row.strip('[]').strip("'").split("', '")

def clean_list_mealType(row):
    return row.strip('[]').strip("'").split("/")

def list_to_str(row):
    return " ".join(row)

def clean_dict(row):
    x = row.strip('[]').split("}, {")
    x = [i.strip('{').split(", '") for i in x]
    xx = [i.split(": ") for i in x[0]]
    return {key.strip("'"):val.strip("'") for key, val in xx}

In [7]:
recipes.drop(columns=['images','shareAs'], inplace=True)

In [8]:
recipes.drop(columns=['ingredients', 'recipe_id'], inplace=True)

In [9]:
recipes['dietLabels'] = recipes['dietLabels'].apply(clean_list)
recipes['healthLabels'] = recipes['healthLabels'].apply(clean_list)
recipes['cautions'] = recipes['cautions'].apply(clean_list)
recipes['cuisineType'] = recipes['cuisineType'].apply(clean_list)
recipes['mealType'] = recipes['mealType'].apply(clean_list_mealType)
recipes['dishType'] = recipes['dishType'].apply(clean_list)

# recipes['ingredients'] = recipes['ingredients'].apply(clean_dict)

In [10]:
dietLabels_labels = list(set([label for row in recipes['dietLabels'] for label in row]))
dietLabels_labels

['Balanced', 'Low-Sodium', 'High-Fiber']

In [11]:
healthLabels_labels = list(set([label for row in recipes['healthLabels'] for label in row]))
healthLabels_labels

['Gluten-Free',
 'Low Potassium',
 'Red-Meat-Free',
 'Egg-Free',
 'Mustard-Free',
 'Tree-Nut-Free',
 'Vegan',
 'Peanut-Free',
 'Alcohol-Free',
 'Dairy-Free',
 'Soy-Free',
 'Sugar-Conscious',
 'Vegetarian',
 'DASH',
 'Wheat-Free',
 'Kidney-Friendly',
 'FODMAP-Free',
 'Shellfish-Free',
 'Pork-Free',
 'Sesame-Free',
 'No oil added',
 'Kosher',
 'Paleo',
 'Mollusk-Free',
 'Crustacean-Free',
 'Lupine-Free',
 'Mediterranean',
 'Pescatarian',
 'Sulfite-Free',
 'Immuno-Supportive',
 'Fish-Free',
 'Celery-Free',
 'Keto-Friendly']

In [12]:
cautions_labels = list(set([label for row in recipes['cautions'] for label in row]))
cautions_labels

['',
 'Eggs',
 'Wheat',
 'Gluten',
 'Shellfish',
 'Milk',
 'Sulfites',
 'Tree-Nuts',
 'Soy',
 'FODMAP']

In [13]:
recipes['cautions'].value_counts()

[Sulfites]                           153
[]                                    36
[Gluten, Wheat, Sulfites]             12
[Sulfites, FODMAP]                     9
[Soy, Sulfites]                        4
[Shellfish, Sulfites, FODMAP]          2
[Wheat, Sulfites]                      2
[Tree-Nuts, Sulfites, FODMAP]          2
[Tree-Nuts]                            2
[Tree-Nuts, Sulfites]                  2
[Gluten, Wheat, Sulfites, FODMAP]      1
[Milk, Sulfites]                       1
[Eggs, Milk, Sulfites]                 1
[FODMAP]                               1
Name: cautions, dtype: int64

In [14]:
cuisineType_labels = list(set([label for row in recipes['cuisineType'] for label in row]))
cuisineType_labels

['south east asian',
 'central europe',
 'asian',
 'chinese',
 'indian',
 'british',
 'italian',
 'greek',
 'french',
 'caribbean',
 'japanese',
 'middle eastern',
 'american',
 'mediterranean',
 'eastern europe']

In [15]:
mealType_labels = list(set([label for row in recipes['mealType'] for label in row]))
mealType_labels

["breakfast', 'lunch",
 'dinner',
 'brunch',
 'breakfast',
 'snack',
 'teatime',
 'lunch']

In [16]:
dishType_labels = list(set([label for row in recipes['dishType'] for label in row]))
dishType_labels

['soup',
 'salad',
 'starter',
 'desserts',
 'cereals',
 'sandwiches',
 'nan',
 'biscuits and cookies',
 'condiments and sauces',
 'main course',
 'pancake',
 'bread',
 'drinks']

In [17]:
# converting into strings
recipes['dishType'] = recipes['dishType'].apply(list_to_str)
recipes['cuisineType'] = recipes['cuisineType'].apply(list_to_str)
recipes['cautions'] = recipes['cautions'].apply(list_to_str)
recipes['healthLabels'] = recipes['healthLabels'].apply(list_to_str)
recipes['dietLabels'] = recipes['dietLabels'].apply(list_to_str)
recipes['mealType'] = recipes['mealType'].apply(list_to_str)


In [18]:
recipes['mealType']

0      lunch dinner
1      lunch dinner
2      lunch dinner
3             snack
4      lunch dinner
           ...     
223    lunch dinner
224    lunch dinner
225    lunch dinner
226    lunch dinner
227    lunch dinner
Name: mealType, Length: 228, dtype: object

In [19]:
for labels in dishType_labels:
    recipes[labels] = recipes['dishType'].str.contains(labels).astype(int)


In [20]:
for labels in mealType_labels:
    recipes[labels] = recipes['mealType'].str.contains(labels).astype(int)

In [21]:
for labels in cuisineType_labels:
    recipes[labels] = recipes['cuisineType'].str.contains(labels).astype(int)

# for labels in cautions_labels:
#     recipes[labels] = recipes['cautions'].str.contains(labels).astype(int)

for labels in healthLabels_labels:
    recipes[labels] = recipes['healthLabels'].str.contains(labels).astype(int)

for labels in dietLabels_labels:
    recipes[labels] = recipes['dietLabels'].str.contains(labels).astype(int)

In [22]:
recipes.isnull().values.any()

False

In [23]:
dummy_table = recipes.iloc[:, 10:]
dummy_table

,soup,salad,starter,desserts,cereals,sandwiches,nan,biscuits and cookies,condiments and sauces,main course,...,Mediterranean,Pescatarian,Sulfite-Free,Immuno-Supportive,Fish-Free,Celery-Free,Keto-Friendly,Balanced,Low-Sodium,High-Fiber
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,1,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,1,0
224,0,0,0,0,0,0,0,0,0,1,...,1,1,1,0,1,0,1,1,1,0
225,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,0,1
226,0,0,0,0,0,0,0,0,0,1,...,0,1,1,0,1,1,0,1,0,0


In [24]:
from sklearn.cluster import KMeans, AgglomerativeClustering

kmeans = KMeans(n_clusters=4)

In [25]:
dummy_table.iloc[:,65:75]

,Fish-Free,Celery-Free,Keto-Friendly,Balanced,Low-Sodium,High-Fiber
0,1,1,0,1,1,0
1,0,1,0,1,0,1
2,1,1,0,1,0,0
3,0,1,0,1,0,0
4,1,1,0,1,0,1
...,...,...,...,...,...,...
223,1,1,0,1,1,0
224,1,0,1,1,1,0
225,1,1,0,1,0,1
226,1,1,0,1,0,0


In [26]:
dummy_table

,soup,salad,starter,desserts,cereals,sandwiches,nan,biscuits and cookies,condiments and sauces,main course,...,Mediterranean,Pescatarian,Sulfite-Free,Immuno-Supportive,Fish-Free,Celery-Free,Keto-Friendly,Balanced,Low-Sodium,High-Fiber
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,1,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,1,0
224,0,0,0,0,0,0,0,0,0,1,...,1,1,1,0,1,0,1,1,1,0
225,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,0,1
226,0,0,0,0,0,0,0,0,0,1,...,0,1,1,0,1,1,0,1,0,0


In [27]:
dummy_table.drop(columns=['nan'], inplace=True)

In [28]:
kmeans.fit(dummy_table)

KMeans(n_clusters=4)

In [29]:
clusters = kmeans.predict(dummy_table)
clusters

array([0, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 3,
       2, 3, 3, 3, 2, 0, 3, 3, 0, 3, 3, 3, 3, 2, 2, 2, 1, 1, 2, 0, 2, 3,
       1, 2, 0, 2, 0, 0, 3, 2, 0, 0, 3, 1, 3, 2, 2, 1, 2, 3, 1, 3, 3, 2,
       3, 3, 3, 0, 2, 2, 3, 3, 3, 2, 2, 2, 3, 0, 3, 0, 3, 1, 3, 1, 2, 2,
       3, 2, 2, 2, 2, 3, 1, 2, 1, 1, 3, 3, 0, 2, 0, 1, 2, 3, 3, 3, 1, 3,
       2, 2, 2, 2, 3, 2, 1, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 0, 3, 2,
       3, 0, 1, 0, 0, 0, 0, 1, 3, 1, 1, 1, 3, 1, 3, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 3, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 3, 3, 0, 1, 3,
       3, 3, 1, 1, 1, 3, 3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 2, 3, 1, 3, 3, 0,
       3, 1, 3, 0, 2, 0, 0, 0, 0, 1, 3, 1, 0, 0, 2, 1, 0, 1, 3, 1, 0, 3,
       1, 2, 1, 2, 3, 1, 0, 0])

In [30]:
final_df = recipes[['label']] 

In [31]:
final_df['clusters'] = clusters

C:\Users\inesa\AppData\Local\Temp\ipykernel_15884\871264172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['clusters'] = clusters


In [32]:
final_df

,label,clusters
0,Easy Chicken Biryani,0
1,Three-Onion Farro Soup,2
2,Broccoli Cheese Soup Recipe,3
3,Sauce for BBQ Lamb Cutlets,2
4,Pesto Chicken Sandwiches recipes,2
...,...,...
223,Strawberry Jello Salad Recipe,2
224,Vegetable Kugel with Matzah Recipe,3
225,Chunky Black Bean Gazpacho,1
226,Spinach and Feta Mashed Potatoes recipes,0


In [36]:
final_df[final_df['clusters'] == 3]

,label,clusters
2,Broccoli Cheese Soup Recipe,3
21,pumpkin waffles,3
23,Sweet Potato Pancakes,3
24,Spaetzle,3
25,Waffles,3
...,...,...
200,Fire Roasted Mac and Cheese with Feta recipes,3
208,Skillet Eggs In Marinara Sauce With Couscous A...,3
216,Pink Potato Soup Recipe,3
219,"Peppers Stuffed with Quinoa, Corn and Feta Cheese",3
